# **Instructions**
1. Data Splitting (only on SWIR)
*   15% to test set (always remains the same, in 1 folder)
*   Remaining 85% to training and validation sets (in 1 folder)

Input folder structure (folder *dataset_20200803*):

    ├───ccd
    ├───flir
    └───swir
        ├───Barge
        ├───BulkCarrier
        ├───ChemicalTanker
        ├───ContainerShip
        ├───Cruise
        ├───Dredger
        ├───Ferry
        ├───GeneralCargoShip
        ├───HarbourLaunch-PilotVessel
        ├───HeavyLoad
        ├───LNG-LPG
        ├───RORO
        ├───Sampan
        ├───SupplyVessel
        ├───Tanker-Bunker
        ├───Trawler-FishingVessel
        ├───Tugboat
        └───Yacht

# Data Splitting (step 1)

Took 15% of the original data to be used as test data, kept in

```
testing_dir
```
Remaining 85% of code will be used for training and validation, kept in

```
other_data_dir
```

In [ ]:
import os
import random
import shutil

data_dir = '/content/drive/MyDrive/2023_internship/dataset_20200803/swir/'  # Path to the folder containing class subfolders
testing_dir = '/content/drive/MyDrive/2023_internship/dataset_20200803/testing_flir2'  # Path to the folder where testing data will be saved
other_data_dir = '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2'  # Path to the folder where remaining data will be saved

# Create testing and other data directories if they don't exist
os.makedirs(testing_dir, exist_ok=True)
os.makedirs(other_data_dir, exist_ok=True)

# Define the percentage of data to use for testing
testing_percentage = 0.15

# Iterate through each class folder
for class_name in os.listdir(data_dir):
    class_folder = os.path.join(data_dir, class_name)

    if os.path.isdir(class_folder):
        # List all files in the class folder
        class_files = os.listdir(class_folder)

        # Calculate the number of testing images
        num_testing = int(len(class_files) * testing_percentage)

        # Randomly select testing images
        testing_files = random.sample(class_files, num_testing)
        for testing_file in testing_files:
            src_path = os.path.join(class_folder, testing_file)
            dest_path = os.path.join(testing_dir, class_name, testing_file)
            os.makedirs(os.path.dirname(dest_path), exist_ok=True)
            shutil.copy(src_path, dest_path)

        # Move other images to the other data directory
        for other_file in class_files:
            if other_file not in testing_files:
                src_path = os.path.join(class_folder, other_file)
                dest_path = os.path.join(other_data_dir, class_name, other_file)
                os.makedirs(os.path.dirname(dest_path), exist_ok=True)
                shutil.copy(src_path, dest_path)

print("Data splitting completed.")


Data splitting completed.


# Data Augmentation (step 2)

Using the remaining 85% data after step 1, data augmentation is done.

Different augmentation methods explored:


1.   Flip
2.   Contrast
3.   Brightening
4.   Sharpening
5.   Flipping and contrasting
6.   Contrasting and sharpening
7.   Flipping and sharpening
8.   Contrasting, sharpening and flipping



## Method 1: Flipping

Code saves **original and flipped images** in a new folder *output_folder*.

In [ ]:
import os
from PIL import Image
from PIL import ImageEnhance
import random
import string
import time

def process_folder(folder_path):
    items = os.listdir(folder_path)

    for item in items:
        item_path = os.path.join(folder_path, item)

        if item_path != '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2/withFlipped': # to prevent going into new folder created
            if os.path.isdir(item_path):
                process_folder(item_path)
            else:
                # If the item is a file, perform specific actions here
                # Assuming it's an image file, you can check for image extensions like '.jpg', '.png', etc.
                if item.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                    file_name, file_extension = os.path.splitext(item)
                    image = Image.open(item_path)
                    lst = folder_path.split("/")
                    # group is the name of the class
                    group = lst[-1]

                    # flipping horizontally
                    flipped_horizontal_img = image.transpose(Image.FLIP_LEFT_RIGHT)
                    saved_path_horizontal = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2/withFlipped', group, f"{file_name}_flipH{file_extension}")
                    horizon_folder = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2/withFlipped', group)

                    # creates horizon_folder if does not exist
                    if not os.path.exists(horizon_folder):
                      os.makedirs(horizon_folder)

                    # saves flipped image
                    flipped_horizontal_img.save(saved_path_horizontal)

                    saved_path_original = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2/withFlipped', group, f"{file_name}_orig{file_extension}")
                    image.save(saved_path_original) # saves original image


# Replace 'your_folder_path' with the path to the folder you want to traverse
your_folder_path = '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2'
output_folder = '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2/withFlipped'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

process_folder(your_folder_path)

Code saves **ONLY** flipped images in a new folder *output_folder*.

In [ ]:
import os
from PIL import Image
from PIL import ImageEnhance
import random
import string
import time

def process_folder(folder_path):
    items = os.listdir(folder_path)

    for item in items:
        item_path = os.path.join(folder_path, item)
        #print(item_path)

        if os.path.isdir(item_path):
            process_folder(item_path)
        else:
            # If the item is a file, perform specific actions here
            # Assuming it's an image file, you can check for image extensions like '.jpg', '.png', etc.
            if item.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                file_name, file_extension = os.path.splitext(item)
                image = Image.open(item_path)
                lst = folder_path.split("/")
                group = lst[-1]
                # flipping horizontally
                flipped_horizontal_img = image.transpose(Image.FLIP_LEFT_RIGHT)
                saved_path_horizontal = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/onlyAugmentedFlir/flipped', group, f"{file_name}_flipH{file_extension}")
                horizon_folder = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/onlyAugmentedFlir/flipped', group)
                #print(saved_path_horizontal)
                if not os.path.exists(horizon_folder):
                  os.makedirs(horizon_folder)
                flipped_horizontal_img.save(saved_path_horizontal)


# Replace 'your_folder_path' with the path to the folder you want to traverse
your_folder_path = '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2'
output_folder = '/content/drive/MyDrive/2023_internship/dataset_20200803/onlyAugmentedFlir/flipped'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


process_folder(your_folder_path)

## Method 2: Contrasting

Code saves **original and contrasted images** in a new folder *output_folder*.

In [ ]:
import os
from PIL import Image
from PIL import ImageEnhance
import random
import string
import time

def process_folder(folder_path):
    items = os.listdir(folder_path)

    for item in items:
        item_path = os.path.join(folder_path, item)

        if item_path != '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2/withContrasted': # to prevent going into new folder created
            if os.path.isdir(item_path):
                process_folder(item_path)
            else:
                # If the item is a file, perform specific actions here
                # Assuming it's an image file, you can check for image extensions like '.jpg', '.png', etc.
                if item.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                    file_name, file_extension = os.path.splitext(item)
                    image = Image.open(item_path)
                    lst = folder_path.split("/")
                    # group is the name of the class
                    group = lst[-1]

                    # contrasting image
                    enhancer = ImageEnhance.Contrast(image)
                    contrasted_img = enhancer.enhance(1.3) # can play around with the contrasting factor
                    saved_path_contrast = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2/withContrasted', group, f"{file_name}_contrast{file_extension}")
                    contrast_folder = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2/withContrasted', group)

                    if not os.path.exists(contrast_folder):
                      os.makedirs(contrast_folder)
                    contrasted_img.save(saved_path_contrast)

                    saved_path_original = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2/withContrasted', group, f"{file_name}_orig{file_extension}")
                    image.save(saved_path_original) # saves original image


# Replace 'your_folder_path' with the path to the folder you want to traverse
your_folder_path = '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2'
output_folder = '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2/withContrasted'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

process_folder(your_folder_path)

Code saves **ONLY** contrasted images in a new folder *output_folder*.

In [ ]:
import os
from PIL import Image
from PIL import ImageEnhance
import random
import string
import time

def process_folder(folder_path):
    items = os.listdir(folder_path)

    for item in items:
        item_path = os.path.join(folder_path, item)
        #print(item_path)

        if os.path.isdir(item_path):
            process_folder(item_path)
        else:
            # If the item is a file, perform specific actions here
            # Assuming it's an image file, you can check for image extensions like '.jpg', '.png', etc.
            if item.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                file_name, file_extension = os.path.splitext(item)
                image = Image.open(item_path)
                lst = folder_path.split("/")
                group = lst[-1]
                # flipping horizontally

                enhancer = ImageEnhance.Contrast(image)
                contrasted_img = enhancer.enhance(1.3) # can play around with the contrasting factor
                saved_path_contrast = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/onlyAugmented/contrasted133_with_AIgen_test2_final', group, f"{file_name}_contrast{file_extension}")
                horizon_folder = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/onlyAugmented/contrasted133_with_AIgen_test2_final', group)
                #print(saved_path_horizontal)
                if not os.path.exists(horizon_folder):
                  os.makedirs(horizon_folder)
                contrasted_img.save(saved_path_contrast)


# Replace 'your_folder_path' with the path to the folder you want to traverse
your_folder_path = '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_swir2'
output_folder = '/content/drive/MyDrive/2023_internship/dataset_20200803/onlyAugmented/contrasted133_with_AIgen_test2_final'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


process_folder(your_folder_path)

## Method 3: Brightening

Code saves **original and brightened images** in a new folder *output_folder*.

SN:
This method was not great, and F1 score did not improve by a lot, so decided not to use this method.

In [ ]:
import os
from PIL import Image
from PIL import ImageEnhance
import random
import string
import time

def process_folder(folder_path):
    items = os.listdir(folder_path)

    for item in items:
        item_path = os.path.join(folder_path, item)
        #print(item_path)

        if item_path != '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withBrightened' and item_path != '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withFlipped':
            if os.path.isdir(item_path):
                process_folder(item_path)
            else:
                # If the item is a file, perform specific actions here
                # Assuming it's an image file, you can check for image extensions like '.jpg', '.png', etc.
                if item.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                    file_name, file_extension = os.path.splitext(item)
                    image = Image.open(item_path)
                    lst = folder_path.split("/")
                    group = lst[-1]
                    # flipping horizontally

                    enhancer = ImageEnhance.Brightness(image)
                    brightened_img = enhancer.enhance(1.2) # can play around with brightening factor
                    saved_path_brighten = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withBrightened', group, f"{file_name}_bright{file_extension}")
                    horizon_folder = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withBrightened', group)
                    #print(saved_path_horizontal)
                    if not os.path.exists(horizon_folder):
                      os.makedirs(horizon_folder)
                    brightened_img.save(saved_path_brighten)
                    saved_path_original = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withBrightened', group, f"{file_name}_orig{file_extension}")
                    #print(saved_path_original)
                    image.save(saved_path_original)


# Replace 'your_folder_path' with the path to the folder you want to traverse
your_folder_path = '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2'
output_folder = '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withBrightened'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


process_folder(your_folder_path)

## Method 4: Sharpening

Code saves **original and sharpened images** in a new folder *output_folder*.

In [ ]:
import os
from PIL import Image, ImageFilter
from PIL import ImageEnhance
import random
import string
import time

def process_folder(folder_path):
    items = os.listdir(folder_path)

    for item in items:
        item_path = os.path.join(folder_path, item)
        #print(item_path)

        if item_path != '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withBrightened' and item_path != '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withFlipped' and item_path != '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withSharpened':
            if os.path.isdir(item_path):
                process_folder(item_path)
            else:
                # If the item is a file, perform specific actions here
                # Assuming it's an image file, you can check for image extensions like '.jpg', '.png', etc.
                if item.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                    file_name, file_extension = os.path.splitext(item)
                    image = Image.open(item_path)
                    lst = folder_path.split("/")
                    group = lst[-1]

                    # sharpening
                    sharpened_image = image.filter(ImageFilter.SHARPEN)
                    saved_path_sharpen = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withSharpened', group, f"{file_name}_sharpen{file_extension}")
                    sharpen_folder = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withSharpened', group)
                    if not os.path.exists(sharpen_folder):
                      os.makedirs(sharpen_folder)
                    sharpened_image.save(saved_path_sharpen)

                    saved_path_original = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withSharpened', group, f"{file_name}_orig{file_extension}")
                    image.save(saved_path_original)


# Replace 'your_folder_path' with the path to the folder you want to traverse
your_folder_path = '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2'
output_folder = '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2/withSharpened'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


process_folder(your_folder_path)

Code saves **ONLY** sharpened images in a new folder *output_folder*.

In [ ]:
import os
from PIL import Image, ImageFilter
from PIL import ImageEnhance
import random
import string
import time

def process_folder(folder_path):
    items = os.listdir(folder_path)

    for item in items:
        item_path = os.path.join(folder_path, item)
        #print(item_path)
        if os.path.isdir(item_path):
            process_folder(item_path)
        else:
            # If the item is a file, perform specific actions here
            # Assuming it's an image file, you can check for image extensions like '.jpg', '.png', etc.
            if item.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                file_name, file_extension = os.path.splitext(item)
                image = Image.open(item_path)
                lst = folder_path.split("/")
                group = lst[-1]

                # sharpening image
                sharpened_image = image.filter(ImageFilter.SHARPEN)
                saved_path_sharpen = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/onlyAugmentedFlir/sharpened', group, f"{file_name}_sharpen{file_extension}")
                sharpen_folder = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/onlyAugmentedFlir/sharpened', group)
                #print(saved_path_horizontal)
                if not os.path.exists(sharpen_folder):
                  os.makedirs(sharpen_folder)
                sharpened_image.save(saved_path_sharpen)

# Replace 'your_folder_path' with the path to the folder you want to traverse
your_folder_path = '/content/drive/MyDrive/2023_internship/dataset_20200803/trng_vali_flir2'
output_folder = '/content/drive/MyDrive/2023_internship/dataset_20200803/onlyAugmentedFlir/sharpened'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


process_folder(your_folder_path)

## Method 5: Contrast + Flip

*your_folder_path* is the folder with **original + contrasted images**.

This snippet of code will **flip** the images found in *your_folder_path*.

*output_folder* will have images of the following permutations:


*   Original
*   Contrasted
*   Flipped
*   Contrasted and flipped

In [ ]:
import os
from PIL import Image
from PIL import ImageEnhance
import random
import string
import time

def process_folder(folder_path):
    items = os.listdir(folder_path)

    for item in items:
        item_path = os.path.join(folder_path, item)
        #print(item_path)

        if os.path.isdir(item_path):
            process_folder(item_path)
        else:
            # If the item is a file, perform specific actions here
            # Assuming it's an image file, you can check for image extensions like '.jpg', '.png', etc.
            if item.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                file_name, file_extension = os.path.splitext(item)
                image = Image.open(item_path)
                lst = folder_path.split("/")
                group = lst[-1]

                # flipping horizontally
                flipped_horizontal_img = image.transpose(Image.FLIP_LEFT_RIGHT)
                saved_path_horizontal = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withContrastedFlipped', group, f"{file_name}_flipH{file_extension}")
                horizon_folder = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withContrastedFlipped', group)
                #print(saved_path_horizontal)
                if not os.path.exists(horizon_folder):
                  os.makedirs(horizon_folder)
                flipped_horizontal_img.save(saved_path_horizontal)
                saved_path_original = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withContrastedFlipped', group, f"{file_name}_orig{file_extension}")
                #print(saved_path_original)
                image.save(saved_path_original)


# Replace 'your_folder_path' with the path to the folder you want to traverse
# your_folder_path should be the folder with the original + contrasted images
your_folder_path = '/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withContrasted'
output_folder = '/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withContrastedFlipped'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


process_folder(your_folder_path)

## Method 6: Contrast + Sharpen

*your_folder_path* is the folder with **original + contrasted images**.

This snippet of code will **sharpen** the images found in *your_folder_path*.

*output_folder* will have images of the following permutations:


*   Original
*   Contrasted
*   Sharpened
*   Contrasted and sharpened

In [ ]:
import os
from PIL import Image, ImageFilter
from PIL import ImageEnhance
import random
import string
import time

def process_folder(folder_path):
    items = os.listdir(folder_path)

    for item in items:
        item_path = os.path.join(folder_path, item)
        #print(item_path)

        if os.path.isdir(item_path):
            process_folder(item_path)
        else:
            # If the item is a file, perform specific actions here
            # Assuming it's an image file, you can check for image extensions like '.jpg', '.png', etc.
            if item.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                file_name, file_extension = os.path.splitext(item)
                image = Image.open(item_path)
                lst = folder_path.split("/")
                group = lst[-1]

                # sharpening
                sharpened_image = image.filter(ImageFilter.SHARPEN)
                saved_path_sharpen = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withContrastedSharpened', group, f"{file_name}_sharpen{file_extension}")
                sharpen_folder = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withContrastedSharpened', group)

                if not os.path.exists(sharpen_folder):
                  os.makedirs(sharpen_folder)
                sharpened_image.save(saved_path_sharpen)
                saved_path_original = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withContrastedSharpened', group, f"{file_name}_orig{file_extension}")

                image.save(saved_path_original)


# Replace 'your_folder_path' with the path to the folder you want to traverse
# your_folder_path should be the folder with original + contrasted images
your_folder_path = '/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withContrasted'
output_folder = '/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withContrastedSharpened'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


process_folder(your_folder_path)

## Method 7: Flip + Sharpen

*your_folder_path* is the folder with **original + flipped images**.

This snippet of code will **sharpen** the images found in *your_folder_path*.

*output_folder* will have images of the following permutations:


*   Original
*   Sharpened
*   Flipped
*   Sharpened and flipped

In [ ]:
import os
from PIL import Image, ImageFilter
from PIL import ImageEnhance
import random
import string
import time

def process_folder(folder_path):
    items = os.listdir(folder_path)

    for item in items:
        item_path = os.path.join(folder_path, item)
        #print(item_path)

        if os.path.isdir(item_path):
            process_folder(item_path)
        else:
            # If the item is a file, perform specific actions here
            # Assuming it's an image file, you can check for image extensions like '.jpg', '.png', etc.
            if item.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                file_name, file_extension = os.path.splitext(item)
                image = Image.open(item_path)
                lst = folder_path.split("/")
                group = lst[-1]
                # sharpening
                sharpened_image = image.filter(ImageFilter.SHARPEN)
                saved_path_sharpen = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withFlippedSharpened', group, f"{file_name}_sharpen{file_extension}")
                sharpen_folder = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withFlippedSharpened', group)

                if not os.path.exists(sharpen_folder):
                  os.makedirs(sharpen_folder)
                sharpened_image.save(saved_path_sharpen)
                saved_path_original = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withFlippedSharpened', group, f"{file_name}_orig{file_extension}")

                image.save(saved_path_original)


# Replace 'your_folder_path' with the path to the folder you want to traverse
# your_folder_path is the folder with original + flipped images
your_folder_path = '/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withFlipped'
output_folder = '/content/drive/MyDrive/2023_internship/dataset_20200803/flir_augmentedData/withFlippedSharpened'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


process_folder(your_folder_path)

## Method 8: Contrast + Flip + Sharpen

*your_folder_path* is the folder of images generated by method 7 (flip + sharpen).

This snippet of code will **contrast** the images found in *your_folder_path*.

*output_folder* will have images of the following permutations:

*   Original
*   Contrasted
*   Flipped
*   Sharpen
*   Contrasted and sharpened
*   Flipped and sharpened
*   Contrasted and flipped
*   Contrasted, flipped and sharpened

In [ ]:
import os
from PIL import Image, ImageFilter
from PIL import ImageEnhance
import random
import string
import time

def process_folder(folder_path):
    items = os.listdir(folder_path)

    for item in items:
        item_path = os.path.join(folder_path, item)
        #print(item_path)

        if os.path.isdir(item_path):
            process_folder(item_path)
        else:
            # If the item is a file, perform specific actions here
            # Assuming it's an image file, you can check for image extensions like '.jpg', '.png', etc.
            if item.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                file_name, file_extension = os.path.splitext(item)
                image = Image.open(item_path)
                lst = folder_path.split("/")
                group = lst[-1]

                # contrast images
                enhancer = ImageEnhance.Contrast(image)
                contrasted_img = enhancer.enhance(1.3)
                saved_path_contrast = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/augmentedData/withFlippedSharpenedContrasted', group, f"{file_name}_contrast{file_extension}")
                contrast_folder = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/augmentedData/withFlippedSharpenedContrasted', group)

                if not os.path.exists(contrast_folder):
                  os.makedirs(contrast_folder)
                contrasted_img.save(saved_path_contrast)

                saved_path_original = os.path.join('/content/drive/MyDrive/2023_internship/dataset_20200803/augmentedData/withFlippedSharpenedContrasted', group, f"{file_name}_orig{file_extension}")
                image.save(saved_path_original)


# Replace 'your_folder_path' with the path to the folder you want to traverse
your_folder_path = '/content/drive/MyDrive/2023_internship/dataset_20200803/augmentedData/withFlippedSharpened'
output_folder = '/content/drive/MyDrive/2023_internship/dataset_20200803/augmentedData/withFlippedSharpenedContrasted'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


process_folder(your_folder_path)

In [ ]:
test_dir = '/content/drive/MyDrive/2023_internship/dataset_20200803/testing_flir2'
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 174 files belonging to 15 classes.


In [ ]:
import tensorflow as tf
import numpy as np

model = keras.models.load_model("/content/drive/MyDrive/flir_model_0.keras")
# Initialize lists to hold images and labels
test_images = []
test_labels = []

# Iterate through the test_ds dataset
for images, labels in test_ds:
    # Iterate through the batch of images and labels
    for img_array, label in zip(images, labels):
        test_images.append(img_array.numpy())  # Convert tensor to numpy array
        test_labels.append(label.numpy())

# Convert lists to arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)